In [1]:
import sys,os
sys.path.insert(0, os.path.join(os.path.abspath(''), "YOLOX"))

import numpy as np
import cv2
import imutils
import time

from tracker import Tracker
from pose_detector import PoseDetector
from sequence_detector import SequenceDetector
from pose_tracker import PoseTracker
from visualizer import Visualizer
from project_utils import identify_bbox

environment = "local"

/home/tgieruc/miniconda3/envs/dlfav/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if environment == "colab":
    from IPython.display import display, Javascript, Image, clear_output
    from base64 import b64decode, b64encode
    import cv2, PIL, io
     # Import PyDrive and associated libraries.
    # This only needs to be done once per notebook.
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    # Authenticate and create the PyDrive client.
    # This only needs to be done once per notebook.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    !gdown --id 1iYxlkO_RZJ_6iL6wdIbfjl8VprLlFp1c

         # function to convert the JavaScript object into an OpenCV image
    def js_to_image(js_reply):
        """
        Params:
                js_reply: JavaScript object containing image from webcam
        Returns:
                img: OpenCV BGR image
        """
        # decode base64 image
        image_bytes = b64decode(js_reply.split(',')[1])
        # convert bytes to numpy array
        jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
        # decode numpy array into OpenCV BGR image
        img = cv2.imdecode(jpg_as_np, flags=1)

        return img

    # function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
    def bbox_to_bytes(bbox_array):
        """
        Params:
                bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
        Returns:
              bytes: Base64 image byte string
        """
        # convert array into PIL image
        bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
        iobuf = io.BytesIO()
        # format bbox into png for return
        bbox_PIL.save(iobuf, format='png')
        # format return string
        bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

        return bbox_bytes
else:
    cap = cv2.VideoCapture(0)

In [3]:
from pose_tracker import PoseTracker
tracker = Tracker('person')
pose_detector = PoseDetector(n_feature=8, n_output=3, checkpoint_path="pose_classification_simplified.ckpt", simple_mode=True, model_based=False)
sequence_detector = SequenceDetector([1,2], 10)
pose_tracker = PoseTracker(pose_detector, sequence_detector, tracker)
visualizer = Visualizer(environment)
poi = None


In [4]:
if environment == "colab":
    # start streaming video from webcam
    video_stream()
    # label for video
    label_html = 'Capturing...'
    # initialze bounding box to empty
    bbox = ''
    #count = 0
    
    
state = 0
poi = None

while True:
    if environment == "colab":
        js_reply = video_frame(label_html, bbox)
        if not js_reply:
            break
        # convert JS response to OpenCV Image
        img = js_to_image(js_reply["img"])
        # create transparent overlay for bounding box
        bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    else:
        _, img = cap.read()
        if img is None:
            break
            
    im = imutils.resize(img, height=500)

    #initialization
    if state == 0:
        outputs, poi = pose_tracker.update(im)
        pose_in_progress = False
        for i,  p in pose_tracker.sequence_detector.people.items():
            if (p.i_seq > 0) and (i != poi):
                pose_in_progress = True
                break
        if (poi != None) and (not pose_in_progress):
            last_detected = time.time()
            state = 1
    if state == 1:
        check_poi = 0
        outputs = np.array(pose_tracker.tracker.update(im))
        for i in range(outputs.shape[0]) :
            if outputs[i,4] ==poi:
                check_poi = 1
        if (check_poi == 0) or (time.time()-last_detected >1):
            state = 0
        
    visualizer.show(im, outputs, poi)

/tmp/pip-install-4y87dran/openpifpaf_6aac84dbeb8e466293e96168b9f10b8d/src/openpifpaf/csrc/src/cif_hr.cpp:102: UserInfo: resizing cifhr buffer
/tmp/pip-install-4y87dran/openpifpaf_6aac84dbeb8e466293e96168b9f10b8d/src/openpifpaf/csrc/src/occupancy.cpp:53: UserInfo: resizing occupancy buffer


/home/tgieruc/DLFAV-PROJECT/pose_detector.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  _, pred = torch.max(F.softmax(self.keypoint_to_pose_net(torch.from_numpy(keypoints.astype(np.float32)).cuda(device))).data, dim=1)
/home/tgieruc/miniconda3/envs/dlfav/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


KeyboardInterrupt: 

In [ ]:
outputs.shape